In [82]:
import pandas as pd
import datetime as dt

#grab flight plan
flight_plan = pd.read_csv(r'C:\Users\cicic\OneDrive\Desktop\Barrios_Project\barrios\iss-data\csv\iss_flight_plan_20220101-20251231.csv')

#query to only use dates where the event is dock and convert it to a list
dock_days_list = flight_plan[flight_plan['event'] == 'Dock']['datedim'].to_list()

#convert list of date strings into a list of datetime objects
dates_list = [dt.datetime.strptime(date, '%m/%d/%Y').date() for date in dock_days_list]

#print the days difference between each date
num_days = pd.Series(dates_list).diff().dt.days.iloc[1:].astype(int).tolist()
print("number of daays til next dock\n",num_days)




number of daays til next dock
 [4, 25, 22, 18, 24, 13, 43, 67, 15, 22, 12, 18, 76, 15, 5, 13, 67, 2, 13, 59, 21, 2, 19, 48, 29, 41, 10, 10, 17, 2, 15, 8, 30, 22, 7, 22, 29, 35, 16, 6, 14, 9, 44, 33, 4, 22, 1, 34, 3, 19, 17, 62, 4, 27, 48, 2, 29, 7, 25, 28, 25, 25]


In [83]:
#grab crew members per dock
crewF = pd.read_csv(r'C:\Users\cicic\OneDrive\Desktop\Barrios_Project\barrios\iss-data\csv\iss_flight_plan_crew_20220101-20251321.csv')

crew_per_dock = []

for i, date in enumerate(dock_days_list):
    crewNum = crewF[crewF['datedim'] == date]['crew_count'].tolist()
    Amount = sum(crewNum)
    crew_per_dock.append(Amount)

#number of crew members on board during each dock
print("number of crew per docking\n",crew_per_dock)


number of crew per docking
 [7, 7, 10, 11, 11, 7, 7, 7, 10, 11, 7, 7, 7, 7, 7, 11, 7, 11, 11, 7, 7, 7, 11, 10, 7, 7, 11, 11, 7, 7, 11, 7, 10, 9, 7, 11, 7, 7, 11, 7, 7, 10, 10, 11, 7, 7, 7, 7, 7, 11, 7, 10, 11, 11, 7, 7, 11, 7, 10, 7, 7, 11, 7]


In [84]:
#grab rates
rater = pd.read_csv(r'C:\Users\cicic\OneDrive\Desktop\Barrios_Project\barrios\iss-data\csv\rates_definition.csv')


#consumable must match a consumable on the rates csv. case sensitive.
consumable = 'Pretreat Tanks'
consumable = 'Air'
#consumable = 'Water'


#sum of generated
sum_generated = sum(rater[(rater['affected_consumable'] == consumable) & (rater['type'] == 'generation')]['rate'].to_list())

#grabs the usage rates of consumable
usage_list = rater[(rater['affected_consumable'] == consumable) & (rater['type'] == 'usage')]['rate'].to_list()

#if units contains 'crew'
crew_check = rater[(rater['affected_consumable'] == consumable) & (rater['type'] == 'usage')]['units'].tolist()
crewIndex = []
for j in crew_check:
    if j.find('Crew') != -1:
        crewIndex.append(True)
    else:
        crewIndex.append(False)

sum_usage_list = []
sum_usage_crew_list = []

for n, value in enumerate (usage_list):
    if crewIndex[n] == True:
        sum_usage_crew_list.append(value)
    else:
        sum_usage_list.append(value)

sum_usage = sum(sum_usage_list)
sum_usage_crew = sum(sum_usage_crew_list)

print(crew_check)
print(usage_list)
print('------------')
print(sum_usage_list)
print(sum_usage_crew_list)
print(sum_usage)
print(sum_usage_crew)





['lbs/Day', 'lbs/Day', 'lbs/EVA', 'lbs/EVA']
[0.75, 0.5, 35.0, 4.1]
------------
[0.75, 0.5, 35.0, 4.1]
[]
40.35
0


In [85]:
import math

#create a list of base rates 
listofrates = [sum_usage] * ((len(dock_days_list)) -1)
listofrates_crew = []
consumable_to_send = []
current_sum = 0
res_list = []

for i in range(0, len(listofrates)):
    listofrates_crew.append(crew_per_dock[i] * sum_usage_crew)

for i in range(0, len(listofrates)):
    res_list.append(((listofrates[i] + listofrates_crew[i]) - sum_generated) * num_days[i])


#rounding
for i, value in enumerate(res_list):
    current_sum+=value
    consumable_to_send.append(math.trunc(current_sum))
    current_sum = current_sum - math.trunc(current_sum)
print(consumable_to_send)
    


[161, 1009, 887, 727, 968, 525, 1735, 2703, 605, 888, 484, 727, 3066, 605, 202, 525, 2703, 81, 524, 2381, 847, 81, 767, 1937, 1170, 1654, 404, 403, 686, 81, 605, 323, 1210, 888, 282, 888, 1170, 1413, 645, 242, 565, 363, 1776, 1331, 162, 887, 41, 1372, 121, 766, 686, 2502, 161, 1090, 1937, 80, 1170, 283, 1009, 1129, 1009, 1009]


In [86]:
##RUN ALL or this wont work
dock_days_list.pop()

readable = pd.DataFrame(
    {'Docking Days': dock_days_list,
     'Amount of Consumable to Send': consumable_to_send
    })

display(readable)

,Docking Days,Amount of Consumable to Send
0,2/17/2022,161
1,2/21/2022,1009
2,3/18/2022,887
3,4/9/2022,727
4,4/27/2022,968
...,...,...
57,9/3/2025,283
58,9/10/2025,1009
59,10/5/2025,1129
60,11/2/2025,1009
